In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import linregress
from scipy.stats.stats import pearsonr
import warnings
%matplotlib inline

In [2]:
# Read-in data as Excel file
vix_data_file = "C:/Users/blake/Desktop/spx_vix_clean.xlsx"

In [3]:
#Convert Excel file to Dataframe
stock_data_df = pd.read_excel(vix_data_file,
sheet_name=0,
header=1,
index_col=False,
keep_default_na=False
)

In [4]:
stock_data_df.head()

,Date,SPX,VIX_1,VIX_2,VIX_3,VIX_4,VIX_5,VIX_6,VIX_7,VIX_8,stdev,1/stdev,SPX_VOLUME,SPX_5day,SPX_tom
0,2009-12-31,1115.1,22.95,25.8,26,26.3,26.35,26.3,26.4,26.35,1.17274,0.852706,636905,NaN,1132.98
1,2010-01-01,1132.98,22.1,24.85,25.25,25.75,25.75,25.75,25.85,25.95,1.28798,0.776412,636905,NaN,1132.98
2,2010-01-04,1132.98,22.1,24.85,25.25,25.75,25.75,25.75,25.85,25.95,1.28798,0.776412,1282633,NaN,1136.52
3,2010-01-05,1136.52,21.55,24.45,24.85,25.45,25.5,25.5,25.55,25.7,1.38666,0.72116,1368386,NaN,1137.14
4,2010-01-06,1137.14,20.8,23.55,24.2,24.95,25.05,25.05,25.1,25.3,1.51139,0.661645,1252015,1130.944,1141.69


In [5]:
#Create new Dataframe, set 'Date' as index, drop column named 'Date' 
time_indexed_stock_df = stock_data_df.set_index(stock_data_df['Date'])
time_indexed_stock_df = time_indexed_stock_df.drop(['Date'], axis=1)
time_indexed_stock_df.head(10)

,SPX,VIX_1,VIX_2,VIX_3,VIX_4,VIX_5,VIX_6,VIX_7,VIX_8,stdev,1/stdev,SPX_VOLUME,SPX_5day,SPX_tom
Date,,,,,,,,,,,,,,
2009-12-31,1115.1,22.95,25.8,26,26.3,26.35,26.3,26.4,26.35,1.17274,0.852706,636905,NaN,1132.98
2010-01-01,1132.98,22.1,24.85,25.25,25.75,25.75,25.75,25.85,25.95,1.28798,0.776412,636905,NaN,1132.98
2010-01-04,1132.98,22.1,24.85,25.25,25.75,25.75,25.75,25.85,25.95,1.28798,0.776412,1282633,NaN,1136.52
2010-01-05,1136.52,21.55,24.45,24.85,25.45,25.5,25.5,25.55,25.7,1.38666,0.72116,1368386,NaN,1137.14
2010-01-06,1137.14,20.8,23.55,24.2,24.95,25.05,25.05,25.1,25.3,1.51139,0.661645,1252015,1130.944,1141.69
2010-01-07,1141.69,20.45,23.15,23.85,24.75,24.85,24.85,24.9,25.1,1.57543,0.634749,1553963,1136.262,1144.98
2010-01-08,1144.98,19.9,22.55,23.4,24.4,24.65,24.6,24.75,24.9,1.71432,0.583323,1508175,1138.662,1146.98
2010-01-11,1146.98,19.5,22.25,23.25,24.15,24.3,24.4,24.5,24.65,1.76048,0.568027,1444997,1141.462,1136.22
2010-01-12,1136.22,20.25,22.8,23.65,24.25,24.45,24.45,24.6,24.75,1.51351,0.660716,2089364,1141.402,1145.68


In [6]:
#Drop Nan rows
final_df = time_indexed_stock_df.dropna()
final_df.head(10)

,SPX,VIX_1,VIX_2,VIX_3,VIX_4,VIX_5,VIX_6,VIX_7,VIX_8,stdev,1/stdev,SPX_VOLUME,SPX_5day,SPX_tom
Date,,,,,,,,,,,,,,
2010-01-06,1137.14,20.8,23.55,24.2,24.95,25.05,25.05,25.1,25.3,1.51139,0.661645,1252015,1130.944,1141.69
2010-01-07,1141.69,20.45,23.15,23.85,24.75,24.85,24.85,24.9,25.1,1.57543,0.634749,1553963,1136.262,1144.98
2010-01-08,1144.98,19.9,22.55,23.4,24.4,24.65,24.6,24.75,24.9,1.71432,0.583323,1508175,1138.662,1146.98
2010-01-11,1146.98,19.5,22.25,23.25,24.15,24.3,24.4,24.5,24.65,1.76048,0.568027,1444997,1141.462,1136.22
2010-01-12,1136.22,20.25,22.8,23.65,24.25,24.45,24.45,24.6,24.75,1.51351,0.660716,2089364,1141.402,1145.68
2010-01-13,1145.68,19.4,22.5,23.45,24.15,24.4,24.4,24.55,24.55,1.77341,0.563884,2110033,1143.110,1148.46
2010-01-14,1148.46,18.95,22,23,23.65,23.95,24,24.15,24.25,1.79731,0.556388,1346436,1144.464,1136.03
2010-01-15,1136.03,19.65,22.55,23.55,24.05,24.3,24.35,24.45,24.45,1.65377,0.604679,2031715,1142.674,1150.23
2010-01-18,1150.23,17.85,21.55,22.75,23.55,23.8,23.8,23.9,24.1,2.11723,0.472314,2031715,1143.324,1150.23


In [8]:
X = final_df[["SPX", "VIX_1", "1/stdev", "SPX_VOLUME", "SPX_5day"]]
y = final_df["SPX_tom"].values.reshape(-1, 1)

print("Shape: ", X.shape, y.shape)

Shape:  (2528, 5) (2528, 1)


In [ ]:
#MODEL STARTS HERE!!!


# Assign the data to X and y
# Note: Sklearn requires a two-dimensional array of values
# so we use reshape to create this

X = spx_21day_df["m21avg"].values.reshape(-1, 1)
y = spx_21day_df["SPX_tomorrow"].values.reshape(-1, 1)

print("Shape: ", X.shape, y.shape)

In [ ]:
x_train = X[:1983]
y_train = y[:1983]
x_test = X[1983:]
y_test = y[1983:]

In [ ]:
#implement linear regression
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x_train,y_train)

In [ ]:
#make predictions and find the rmse
preds = model.predict(x_test)
rms=np.sqrt(np.mean(np.power((np.array(y_test)-np.array(preds)),2)))
rms

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
r2 = r2_score(y_test, preds)
r2

In [ ]:
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

#plot
valid['Predictions'] = 0
valid['Predictions'] = preds

valid.index = spx_21day_df[1983:].index
train.index = spx_21day_df[:1983].index

plt.plot(train['SPX_tomorrow'])
plt.plot(valid[['SPX_tomorrow', 'Predictions']])

In [ ]:
predicted_df = spx_21day_df[1983:].copy()

predicted_df['Predicted_tomorrow']=preds

predicted_df.head()

In [ ]:
predicted_df['today'] = spx_df.loc['2017-09-05':]
predicted_df.head(10)

In [ ]:
predicted_df['predicted_delta'] = predicted_df['Predicted_tomorrow']-predicted_df['today']
predicted_df['actual_delta'] = predicted_df['SPX_tomorrow']-predicted_df['today']
predicted_df.head(25)

In [ ]:
pl5_df = predicted_df.filter(['today'], axis=1) 
pl5_df['predicted_delta'] = predicted_df['predicted_delta']
pl5_df.head(10)

In [ ]:
#Calc profit
#Note: only strategy is LONG

pr=0
prt=[]
prt.append(0)
totpr=0
buy=0
sell=0
c_df=pl5_df["today"]
sig_df=pl5_df["predicted_delta"]
for i in range(1,len(c_df)):
   if (sig_df[i-1]<0 and sig_df[i]>0):
       buy=1;
       bp=c_df[i];
   if (sig_df[i-1]>0 and sig_df[i]<0):
       sp=c_df[i];
       if (buy==1):
           pr=sp-bp;
           buy=0;
           totpr+=pr
   prt.append(totpr)
print(prt) 

In [ ]:
fig, ax = plt.subplots()
#fig.suptitle("SPX Predictions", fontsize=16, fontweight="bold")
ax.set_xlabel("Date")
ax.set_ylabel("Profit")
#ax.plot(predicted_df.index, predicted_df["SPX_tomorrow"], '.b')
ax.plot(predicted_df.index, prt, '--r')
plt.show()

In [ ]:
min(prt)